# Assessment 2: Path Following and Collision Avoiding
### Intro
This notebook is to be the record of completion for Assessment 2: Programming JetBots.
### Scenario
Building on the previous pest identification project, program a bot that can both follow a set course and respond when specific pests are located.

In [ ]:
# Pre-setup
# %pip install -Uqq ipywidgets
# %pip install -Uqq torch
# %pip install -Uqq cv2
# %pip install -Uqq jetbot
# %pip install -Uqq numpy

## Path Following
#### Data Collection
The following section is the code and widgets I ran on the JetBot to gather the images required to train the bot in path following.

In [ ]:
## RUN ON JETBOT ##
# Imports for data collection
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from jupyter_clickable_image_widget import ClickableImageWidget

DATASET_DIR = 'dataset_xy'

if not DATASET_DIR.exists():
    os.makedirs(DATASET_DIR)

camera = Camera()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
# manually update counts at initialization
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        #dataset.save_entry(category_widget.value, camera.value, x, y)
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

display(data_collection_widget)

The next block stops the camera once data collection is done, then saves the images to a zip file so it can be exported to a more powerful machine to create the model.

In [ ]:
## RUN ON JETBOT ##
camera.stop()

!zip -r -q path_following_dataset.zip {DATASET_DIR}

#### Training Path Following
The following code was used to train the model based on the images collected, to teach the bot how to follow the path set out. I used resnet18 for the path following model due to its ubiquity and ease of use, for quick and consistent results.

In [ ]:
# Run first to unzip dataset
!unzip -q path_following_dataset.zip

In [ ]:
# Imports for training model
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

def get_x(path, width):
    """Gets the x value from the image filename"""
    return (float(int(path.split("_")[1])) - width/2) / (width/2)

def get_y(path, height):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) - height/2) / (height/2)

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        width, height = image.size
        x = float(get_x(os.path.basename(image_path), width))
        y = float(get_y(os.path.basename(image_path), height))
      
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)
model = models.resnet18(pretrained=True)

model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

NUM_EPOCHS = 40
BEST_MODEL_PATH = 'path_following.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
print("finished")

## Adding Pest Detection
#### Combining Path Following and Bug Identification
To combine both models, I set up two folders, one labeled 'free' with images collected when training the path following, and one labeled 'blocked' with images of insects from the bug finding project. The new model combines both of these datasets to allow the JetBot to follow the path and check for insects, and will stop when it detects an insect matching one it was trained on. The following code is importing the previously trained models to set up this new model.

In [ ]:
# Importing models for path following and bug detection
path_model = torchvision.models.resnet18(pretrained=False)
path_model.fc = torch.nn.Linear(512, 2)
path_model.load_state_dict(torch.load('path_following.pth'))

bug_model = torchvision.models.alexnet(pretrained=False)
bug_model.classifier[6] = torch.nn.Linear(bug_model.classifier[6].in_features, 2)
bug_model.load_state_dict(torch.load('bug_finder.pth')) # how to export as .pth?

device = torch.device('cuda')
path_model = path_model.to(device)
bug_model = bug_model.to(device)

The following segment of code was run on the JetBot to refine the parameters used by testing impacts of changing them in real time. This allowed me to set optimal parameters for the next stage of the process.

In [ ]:
## RUN ON JETBOT ##
camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
robot = Robot()
#Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='Steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='Steering dgain')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='Steering bias')
display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)
#Collision Avoidance sliders
blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='Blocked')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='Time to stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='Blocked threshold')
display(image_widget)
display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

In [ ]:
angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = speed_control_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
    image_preproc2 = preprocess_col(change['new']).to(device)
     
    # Bug Detection model:
    
    prob_blocked = float(F.softmax(bug_model(image_preproc2), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
            xy = path_model(image_preproc2).detach().float().cpu().numpy().flatten()        
            x = xy[0]            
            y = (0.5 - xy[1]) / 2.0
            speed_value = speed_control_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero (can set to turn as well)
        else:
            go_on = 1
            count_stops = 0
                
    angle = math.atan2(x, y)        
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steer_val = pid + steer_bias 
    angle_last = angle
    robot.left_motor.value = max(min(speed_value + steer_val, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_value - steer_val, 1.0), 0.0) 

execute({'new': camera.value})

In [ ]:
# Run bot using model
camera.observe(execute, names='value')

In [ ]:
# Stop bot
camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

## Video Demonstration
**** Insert video here ****


You must provide a short (less than 1-minute) video demonstrating the Jetbot in action (using Xbox game bar recording or simlilar). The video should showcase the path following and pest detection aspects.

## Critical Evaluation
You must critically evaluate the accuracy and effectiveness of your trained model for pest detection during infernece.
You must offer insightful comments on areas where improvements can be made, such as model accuracy, inference speed, and generalizability.

### **Road Following**   
- Upload the "*best_steering_model_xy_trt.pth*" model file obtained from the "live_demo_build_trt.ipynb" into this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy_trt.pth`` in this notebook's directory.    

### **Collision Avoidance**             
- Upload the "*best_model_trt.pth*" model file obtained from the "live_demo_resnet18_build_trt.ipnb" into this notebooks's directory. Once that's finished there should be a file named ``best_model_trt.pth`` in this notebook's directory.   
> **Note**: Collision Avoidance ``blocked`` class should include images of cars or any obstacle which you would expect to see on the road (It's good to start with one object first) meanwhile the ``free`` class should include background images of the road following track. 

Load the TRT optimized models by executing the cell below

In [ ]:
path_model = torchvision.models.resnet18(pretrained=False)
path_model.fc = torch.nn.Linear(512, 2)
path_model.load_state_dict(torch.load('best_steering_model_xy.pth')) # well trained road following model

bug_model = torchvision.models.alexnet(pretrained=False)
bug_model.classifier[6] = torch.nn.Linear(bug_model.classifier[6].in_features, 2)
bug_model.load_state_dict(torch.load('best_model.pth')) # well trained collision avoidance model

In [ ]:
device = torch.device('cuda')
path_model = path_model.to(device)
bug_model = bug_model.to(device)

### Creating the Pre-Processing Function

We have now loaded our models, but there's a slight issue. The format that we trained our models on doesn't exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def preprocess_col(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [ ]:
camera = Camera()

In [ ]:
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

We'll also create our robot instance which we'll need to drive the motors.

In [ ]:
robot = Robot()

Now, we will define some sliders to control the JetBot
> **Note**: We have initialized the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control slider: To start your JetBot increase ``speed_control_slider`` 
2. Steering Gain slider: If you see your JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias slider: If you see your JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

4. Blocked slider: Display the probability in which there is an obstacle in the front of the Jetbot using the collision avoidance model
5. Time for stop slider: To manually set the time for which the jetbot should remain stopped after an object has been removed
6. Blocked threshold slider: To manually set the blocked threshold to stop the Jetbot after an object has been detected 

In [ ]:
#Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

#Collision Avoidance sliders
blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

display(image_widget)

display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network models for Road following and Collision Avoidance
3. Check an if statements which would allow the Jetbot to perform road following and stop whenever an obstacle has been detected 
4. Compute the approximate steering value
5. Control the motors using proportional / derivative control (PD)

In [ ]:
angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = speed_control_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
    image_preproc2 = preprocess_col(change['new']).to(device)
     
    #Collision Avoidance model:
    
    prob_blocked = float(F.softmax(bug_model(image_preproc2), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
            xy = path_model(image_preproc2).detach().float().cpu().numpy().flatten()        
            x = xy[0]            
            y = (0.5 - xy[1]) / 2.0
            speed_value = speed_control_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero (can set to turn as well)
        else:
            go_on = 1
            count_stops = 0
            
    
    angle = math.atan2(x, y)        
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steer_val = pid + steer_bias 
    angle_last = angle
    robot.left_motor.value = max(min(speed_value + steer_val, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_value - steer_val, 1.0), 0.0) 

execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower and collision avoider should work, but the neural network is only as good as the data it's trained on!

In [ ]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow the track and avoid collisions effectively.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

### Training the Jetbot:

You are required to use the NVIDIA Jetbot to train it to follow a predefined path within a specified environment, in this case the classroom following the pink tape.
You must document the entire training process, including code, model selection, and training parameters, in a Jupyter notebook.
The Jupyter notebook should include comments explaining the rationale behind their choices and any issues they encountered.
### Inference for Pest Detection:

Using the model you have created in Assessment 1 (or a similar one), you must implement inference to detect pests.
You need to demonstrate how the Jetbot can identify and respond to pests in real-time during its path following.
The Jupyter notebook should contain code for inference and comments explaining the model's performance.